In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torch.optim import Adam
import torch.nn as nn
import pandas as pd
import os
from PIL import Image
import numpy as np

In [2]:
# Load the CSV file into a DataFrame
csv_file = './partitioned.csv'  # Update this to the correct path
attributes_df = pd.read_csv(csv_file)
attributes_df

,image_id,Gender,partition
0,039088.jpg,Female,0
1,030894.jpg,Male,0
2,045279.jpg,Female,0
3,016399.jpg,Female,0
4,013654.jpg,Male,0
...,...,...,...
49995,035413.jpg,Male,2
49996,013543.jpg,Female,2
49997,010990.jpg,Female,2
49998,027439.jpg,Female,2


In [3]:
# Map filenames to labels (assuming gender is the attribute for classification)
# Let's say 'Male' is represented as 1 and 'Female' as -1 in the CSV
filename_to_label = {row['image_id']: 1 if row['Gender'] == "Male" else 0 for index, row in attributes_df.iterrows()}
filename_to_label

{'039088.jpg': 0,
 '030894.jpg': 1,
 '045279.jpg': 0,
 '016399.jpg': 0,
 '013654.jpg': 1,
 '013749.jpg': 0,
 '023966.jpg': 0,
 '045553.jpg': 0,
 '030220.jpg': 0,
 '024080.jpg': 1,
 '012182.jpg': 0,
 '021849.jpg': 0,
 '014291.jpg': 1,
 '024756.jpg': 0,
 '035014.jpg': 0,
 '015362.jpg': 1,
 '048039.jpg': 1,
 '036312.jpg': 0,
 '030240.jpg': 1,
 '007196.jpg': 0,
 '003580.jpg': 1,
 '036148.jpg': 1,
 '034096.jpg': 0,
 '010442.jpg': 0,
 '016308.jpg': 0,
 '004205.jpg': 0,
 '006958.jpg': 1,
 '009164.jpg': 0,
 '020628.jpg': 1,
 '017568.jpg': 0,
 '014833.jpg': 1,
 '027030.jpg': 0,
 '044198.jpg': 0,
 '000002.jpg': 0,
 '035450.jpg': 0,
 '030904.jpg': 0,
 '007649.jpg': 1,
 '004218.jpg': 0,
 '010475.jpg': 1,
 '001186.jpg': 0,
 '046702.jpg': 0,
 '026827.jpg': 1,
 '012091.jpg': 1,
 '005387.jpg': 1,
 '017352.jpg': 1,
 '018845.jpg': 1,
 '028817.jpg': 0,
 '043053.jpg': 0,
 '011348.jpg': 0,
 '042803.jpg': 0,
 '003980.jpg': 1,
 '007091.jpg': 1,
 '021600.jpg': 1,
 '034265.jpg': 0,
 '017931.jpg': 0,
 '041959.j

In [4]:

def getImagePath(image_id):
    return os.path.join('img_align_celeba',image_id)

df = pd.read_csv("partitioned_multi_attr.csv")
train_df = df[df['partition'] == 0]
val_df = df[df['partition'] == 1]
test_df = df[df['partition'] == 2]

df_labels = df.set_index('image_id')
file_paths = df['image_id'].apply(getImagePath).tolist()


# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [5]:
class CelebADataset(Dataset):
    def __init__(self, file_paths, file_to_label, transform=None):
        self.file_paths = file_paths
        self.file_to_label = file_to_label
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_name = self.file_paths[idx]
        image = Image.open(img_name).convert('RGB')
        label = self.file_to_label[os.path.basename(img_name)][0]  # Only the "Male" label

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
# # Load the CelebA dataset from a single directory
# image_directory = './processed_img'  # Replace with your dataset path
# image_paths = [os.path.join(image_directory, img) for img in os.listdir(image_directory)]

In [7]:
# # Create the dataset
# celeba_dataset = CelebADataset(image_paths, filename_to_label, transform=transform)

# # Now you can create DataLoaders for training and validation
# train_size = int(0.8 * len(celeba_dataset))
# val_size = len(celeba_dataset) - train_size
# train_dataset, val_dataset = torch.utils.data.random_split(celeba_dataset, [train_size, val_size])

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)

In [8]:
# Assuming the images are in a directory named 'images' in the current working directory
# Create separate file path and label mappings for each dataset partition
train_file_paths = train_df['image_id'].apply(getImagePath).tolist()
val_file_paths = val_df['image_id'].apply(getImagePath).tolist()
test_file_paths = test_df['image_id'].apply(getImagePath).tolist()

train_filename_to_label = {filename: labels.values for filename, labels in train_df.set_index('image_id').iterrows()}
val_filename_to_label = {filename: labels.values for filename, labels in val_df.set_index('image_id').iterrows()}
test_filename_to_label = {filename: labels.values for filename, labels in test_df.set_index('image_id').iterrows()}

# Initialize the datasets for each partition
train_dataset = CelebADataset(train_file_paths, train_filename_to_label, transform=transform)
val_dataset = CelebADataset(val_file_paths, val_filename_to_label, transform=transform)
test_dataset = CelebADataset(test_file_paths, test_filename_to_label, transform=transform)

# Create data loaders for each dataset partition
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [9]:
# Load a pre-trained ResNet model
model = resnet50(pretrained=True)

# Modify the model for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Output one value for binary classification

C:\Users\hunts\anaconda3\envs\NN_Proj\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hunts\anaconda3\envs\NN_Proj\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
device

device(type='cuda', index=0)

In [11]:
# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss() 
optimizer = Adam(model.parameters(), lr=0.001)

In [18]:
from tqdm import tqdm

# Early stopping parameters
patience = 5  # How many epochs to wait after last time validation loss improved.
best_loss = np.Inf
epochs_no_improve = 0
early_stop = False

num_epochs = 100  # You can adjust the number of epochs
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    train_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (inputs, labels) in train_bar:
        inputs, labels = inputs.to(device), labels.to(device).float()
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.view(-1)  # Flatten the output to match the labels' shape
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        train_bar.set_description(f'Epoch {epoch+1}/{num_epochs} [Train]')

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    val_bar = tqdm(val_loader, total=len(val_loader))
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()  # Cast labels to Long
            outputs = model(inputs)
            outputs = outputs.view(-1)  # Flatten the output
            loss = criterion(outputs, labels.float())  # Cast labels back to Float for loss calculation
            val_loss += loss.item()
    
            # Apply threshold to get predictions
            predicted = outputs > 0.5  # Using 0.5 as the threshold for binary classification
            predicted = predicted.long()  # Cast predicted to Long to match labels type
    
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_bar.set_description(f'Epoch {epoch+1}/{num_epochs} [Val]')
    
    # Calculate average losses
    train_loss = running_loss / len(train_loader)
    val_loss = val_loss / len(val_loader)
    
    # Print training/validation statistics 
    print(f'Epoch: {epoch+1} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {val_loss:.6f}')
    print(f'Validation Accuracy: {100 * correct / total}%')

    # Save model if validation loss has decreased
    if val_loss < best_loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        best_loss,
        val_loss))
        torch.save(model.state_dict(), 'gender_classification_model.pth')
        best_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping')
            early_stop = True
            break

    if early_stop:
        print("Stopped early due to no improvement in validation loss")
        break

print('Training complete')

Epoch 1/100 [Val]:   0%|                                                                       | 0/157 [00:09<?, ?it/s]

Epoch: 1 	Training Loss: 0.073770 	Validation Loss: 0.094534
Validation Accuracy: 96.82%
Validation loss decreased (inf --> 0.094534).  Saving model ...



Epoch 2/100 [Train]: 100%|█████████████████████████████████████████████████████████| 1250/1250 [02:08<00:00,  9.70it/s]

Epoch 1/100 [Val]:   0%|                                                                       | 0/157 [02:18<?, ?it/s]

Epoch 2/100 [Val]:   0%|                                                                       | 0/157 [00:09<?, ?it/s]

Epoch: 2 	Training Loss: 0.062172 	Validation Loss: 0.079552
Validation Accuracy: 97.26%
Validation loss decreased (0.094534 --> 0.079552).  Saving model ...


Epoch 3/100 [Val]:   0%|                                                                       | 0/157 [00:08<?, ?it/s]

...

Epoch: 8 	Training Loss: 0.027558 	Validation Loss: 0.074181
Validation Accuracy: 97.5%


Epoch 9/100 [Val]:   0%|                                                                       | 0/157 [00:10<?, ?it/s]

Epoch: 9 	Training Loss: 0.023269 	Validation Loss: 0.117772
Validation Accuracy: 95.92%



Epoch 10/100 [Train]: 100%|████████████████████████████████████████████████████████| 1250/1250 [03:00<00:00,  6.93it/s]

Epoch 9/100 [Val]:   0%|                                                                       | 0/157 [03:11<?, ?it/s]

Epoch 10/100 [Val]:   0%|                                                                      | 0/157 [00:10<?, ?it/s]

Epoch: 10 	Training Loss: 0.022861 	Validation Loss: 0.080568
Validation Accuracy: 97.6%
Early stopping
Training complete


In [25]:
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def test_model(model, val_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Evaluating', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            outputs = outputs.view(-1)
            loss = criterion(outputs, labels.float())  # Calculate loss
            val_loss += loss.item()

            # Apply sigmoid since BCEWithLogitsLoss includes the sigmoid layer
            predictions = torch.sigmoid(outputs) > 0.5
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    # Calculate average loss
    avg_loss = val_loss / len(val_loader)

    # Convert to binary values and calculate metrics
    all_predictions = [int(pred) for pred in all_predictions]
    all_targets = [int(target) for target in all_targets]

    # Calculate metrics using sklearn
    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions)
    recall = recall_score(all_targets, all_predictions)
    f1 = f1_score(all_targets, all_predictions)

    return avg_loss, accuracy, precision, recall, f1

# Example usage:
criterion = torch.nn.BCEWithLogitsLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
avg_loss, accuracy, precision, recall, f1 = test_model(model, val_loader, criterion, device)
print(f'Validation Loss: {avg_loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1: {f1:.4f}')

Validation Loss: 0.0694
Accuracy: 0.9754
Precision: 0.9707
Recall: 0.9711
F1: 0.9709
